# Homework 3 Function-based RL
Ponwalai Chalermwattanatrai 65340500042

## Part 1: Understanding the Algorithm

For each algorithm, describe whether it follows a value-based, policy-based, or Actor-Critic approach, specify the type of policy it learns (stochastic or deterministic), identify the type of observation space and action space (discrete or continuous), and explain how each advanced RL method balances exploration and exploitation.

#### 1.Linear Q-Learning

- **Approach**: Value-based
- **Policy type**: Deterministic (Stochastic during training)
    - Always chooses the action with the maximum Q-value (argmax Q(s, a)) -> Deterministic
    - During training, using epsilon-greedy, which includes some random -> Stochastic
- **Observation space**: continuous
- **Action space**: discrete
- **Balances exploration and exploitation**: using epsilon-greedy
    - Selects a random action with probability epsilon and decay epsilon over time

**Concept of algorithm**

Linear Q-learning is a Q-learning algorithm that use function approximation instead of a Q-table. It stores a weight matrix, where each column corresponds to the weights for a particular action. The size of the weight matrix is [state_size x num_actions].

In Linear- Q learning Q-value is linear function calculated using the dot product between the state vector and the weight vector for the selected action:

$$Q(s, a) = \mathbf{w}_a^\top \mathbf{s}$$

After get action and apply to environment, to update the weights, we calculate the Temporal Difference (TD) error using the Bellman equation:

$$\delta = r + \gamma \max_{a'} Q(s', a') - Q(s, a)$$

and define loss function as the mean squared error of TD error (1/2 is Constant that add for easier differential equation which is not impact to value):

$$\mathcal{L} = \frac{1}{2} \delta^2$$

To minimize this loss, we apply gradient descent. The gradient of the loss with respect to the weights is:

$$\nabla_{\mathbf{w}_a} \mathcal{L} = -\delta \cdot \mathbf{s}$$

Therefore, the weights are updated using the following rule:

$$\mathbf{w}_a \leftarrow \mathbf{w}_a + \alpha \cdot \delta \cdot \mathbf{s}$$

Which can be written as:

$$\boxed{
\mathbf{w}_a^{\text{new}} = \mathbf{w}_a^{\text{old}} + \alpha \left( r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right) \cdot \mathbf{s}
}$$

**Psuedo code of training Linear Q learning:**

#### 2. DQN (Deep Q-Learning)

- **Approach**: Value-based
- **Policy type**: Deterministic (Stochastic during training)
    - Always chooses the action with the maximum Q-value (argmax Q(s, a)) -> Deterministic
    - During training, using epsilon-greedy, which includes some random -> Stochastic
- **Observation space**: continuous
- **Action space**: discrete
- **Balances exploration and exploitation**: using epsilon-greedy
    - Selects a random action with probability epsilon and decay epsilon over time

**Concept of algorithm**

DQN (Deep Q-Network) is conceptually similar to Linear Q-learning, but instead of using a linear function for function approximation, DQN uses a neural network to approximate the Q-value function.

Structure of DQN

![image2.png](img/image2.png)

source: https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae

DQN has 3 main components:
1. replay memory
    - A buffer used to store the agent’s past experiences as tuples:(state, action, reward, next state)
    - Using for traning, during training it randomly sample mini-batches of experiences from the buffer to update policy net.
    - mini-batches will breaks the temporal correlation between consecutive experiences and improves stability and sample efficiency.

2. Policy net
    - A neural network that approximates the Q-value.
    - Similar to the weight vector in Linear Q-learning but with hidden layers for greater function approximation.
    - The network is updated using TD error and gradient descent:
    $$\delta = r + \gamma \max_{a'} Q(s', a') - Q(s, a)$$
    - The loss function is the smooth L1 loss (Huber loss):
    $$
    \mathcal{L} =
    \begin{cases}
    \frac{1}{2} \delta^2 & \text{if } |\delta| < 1 \\
    |\delta| - \frac{1}{2} & \text{otherwise}
    \end{cases}
    $$
    - This behaves like MSE for small TD errors and like MAE for large errors — making it less sensitive to outliers.

3. Target net
    - A copy of the policy network that is used to calculate the Q-value for the next state.
    - It prevents the TD target from shifting too quickly, which can destabilize training.
    - Rather than updating the target network every step, DQN uses a soft update mechanism.
    $$\theta_{\text{target}} \leftarrow \tau \cdot \theta_{\text{policy}} + (1 - \tau) \cdot \theta_{\text{target}}$$
    Where:
    - $\theta_{\text{policy}}$​: weights of the policy network
    - $\theta_{\text{target}}$​: weights of the target network
    - $\tau$: soft update rate (between [0,1])
    
    Soft update is prevents rapid oscillations in learning
    
**The relationship of 3 component is**

![image3.png](img/image3.png)

source: https://www.theengineeringprojects.com/2024/01/deep-q-networks-dqn-reinforcement-learning.html

**Psuedo code of training DQN:**

#### 3. MC REINFORCE

- **Approach**: Policy-based
- **Policy type**: Stochastic
    - Learns a probability distribution over actions (Categorical distribution)
- **Observation space**: continuous
- **Action space**: discrete
- **Balances exploration and exploitation**: using stochastic policy
    - In MC REINFORCE action sample from probability distribution, this means the agent naturally explores, since each action has some non-zero probability of being selected.
    - Exploit start when policy increases the probability of high-reward actions through policy gradient updates

**Concept of algorithm**

MC REINFORCE is a Monte Carlo method which using policy gradient to approximate policy. It using complete return of the entire episode from current policy to update new policy.
MC REINFORCE is a Monte Carlo policy gradient method that learns a stochastic policy by using complete returns from sampled episodes. It updates the policy by increasing the probability of actions that lead to high rewards.

Action Selection:
- MC REINFORCE is policy-based, meaning it directly models the policy $\pi(a∣s)$ as a neural network (policy net).
- Given a state s, the policy net outputs a probability distribution over actions.
- A Categorical distribution is constructed from these probabilities, and an action is sampled from it

Policy Update
- In policy update we want to maximize the expected return:
$$J(\theta) = \mathbb{E} [ \sum_{t=0}^{T} r_t ]$$
- Using the episodic policy gradient theorem, we get a formula for how to change the policy parameters $\theta$ to increase $J(\theta)$:
$$\nabla_\theta J(\theta) = \mathbb{E} [\sum_{t=0}^{T} \nabla_\theta \log \pi_\theta(a|s) \cdot G_t ]$$
- So we need to minimize $-\nabla_\theta J(\theta)$, so we give loss function = $-\nabla_\theta J(\theta)$:
$$\mathcal{L} = -\nabla_\theta J(\theta) = - \sum_{t=0}^{T} \log \pi_\theta(a_t \mid s_t) \cdot G_t$$
- when we minimize loss function is equal to maximize policy gradient

from equation, to update the policy, we need:
1. Returns $G_t$​ at each timestep:
    - Calculate from the total discounted future rewards from time t: 
    $$G_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... = r_t + \gamma G_{t+1}​$$
2. Log-probabilities of selected actions:
    - For each timestep, we store:
    $$log\pi(a_t​∣s_t​)$$
    - This term gives us a score function gradient, which tells us how the log-probability of the action changes with respect to the policy parameters.


**Psuedo code of training MC REINFORCE:**